# Secondary Index Ver 6.1+

## Key Features: 
### Revised SI layout in memory
 * EE uses Linux Shared RAM.  Allows for Warm Start of SIs.
 * Revised B-Tree model, one B-tree per partition. 
 * One unique 14 byte entry per record indexed.
 * Indexing Bin Data (same as before) - _types_ supported: Numeric, String and Geo2DSphere
 * CDTs: Indexing Top level Map and Lists data types. MapKeys, MapValues or ListValues of above _types_ can be indexed.
 * 6.1 extends above top-level only indexing capability of CDTs to any context or nested path in a CDT. (Pre 6.1 only top-level bin data containg a map or list data type could be indexed.)
 
 Partition based layout allows query to progress with partition awareness. Allows users to have fine control over query progress. Maintains query progress through cluster changes. No need to track "failOnClusterChange" events, store and delay consumption of results in clients, and restart query in case of cluster change.
 
 <img src="./graphics/SIOverview.png"
     alt="SIOverview"
     style="center; margin-right: 10px;"
     width="800"
     height="640"/>


## Presentation Outline 
* Quick review of the Jupyter Notebook Environment
* Interactive Java Development Environment
* Connecting to Aerospike One node cluster 
* SI 6.1 Features



### Jupyter Notebook Overview
What is happening behind the scenes...

![fig2](./graphics/jpynb_BlockDiag.png)

We are using the iJava (Interactive Java Kernel for Jupyter Notebook) in this example.

We also have the option to code in Python - in a separate Notebook, iPython is the other interactive Kernel available in Jupyter Notebooks.

##### Jupyter Notebook - Cells and Execution / Edit shortcuts 

We have a mix of Markdown cells (text description in Markdown) and Code cells (we have chosen iJava Kernel).
<img src="./graphics/JupNb_Cells.png"
     alt="Fraud Detection"
     style="center; margin-right: 10px;"
     width="250"
     height="400"/>


You can use the menu above to change a cell from being a Code cell to a Markdown cell.

**Cntrl-Enter** executes a cell. Try it.

Clicking anywhere in text display (Markdown) cell and hitting **Enter**, puts you in the cell edit mode. Try it.

Code cells can be edited directly. To run an edited code cell, just hit **Cntrl-Enter**.

You can add a new blank cell, cells can be deleted, copied, pasted or moved up or down using the menu icons as shown below:

![fig4](./graphics/JupNb_CellEditing.png)

### Housekeeping - Wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using aql.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [136]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Runing aql in iJava
We can run aql commands in a file. No output will be displayed. 
In _trucate_test.aql_, we have a single AQL command: TRUNCATE test



In [170]:
%sh aql -h 127.0.0.1 -f "./aqlScripts/truncate_test.aql"

### Runing aql via Terminal Tab

Alternatively, you can also open a separate terminal window (TAB) from the Home Page and run aql interactively in the Terminal.  Please **DO NOT CLICK ON QUIT** on Login Page. It will kill the Jupyter Notebook Server.

![fig5](./graphics/JupNb_Terminal.png)

### Get our Aerospike Java Client talking to the Single Node Aerospike Server 

We will be discussing the data model development interactively. We will code as we progress through the discussion. So let us connect our client application's interactive coding environment with the single node Aerospike server. This will get our real time interaction started.

As we write code in following Jupyter Notebook cells, it will be executed on the Single node Aerospike Server. 

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [138]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>6.1.0</version>
  </dependency>
</dependencies>

####  Add required Java Client Imports

These are some of the Aerospike Java Client imports needed to start developing our Application interactively.  We will add others, as needed, as we develop our solution.

In [139]:
//Require Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
System.out.println("Client modules imported.");

Client modules imported.


#### Connect to the Aerospike Server
Instantiate the client object. Let us write a record and read it back.
We have a namespace **_test_** pre-defined on the server.

In [140]:
AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Key key = new Key("test", "demo", "key1");
System.out.println("Working with record key:");
System.out.println(key);  //Will show the 20 byte digest

Initialized the client and connected to the cluster.
Working with record key:
test:demo:key1:ec91192d4b7f8ce35d5d78d34bca65cbaaaac960


#### Test Record Data Model

![fig_DM_1](./graphics/Fraud_DM_1.png)

In [141]:
//Insert a record with two bins, b1 with string v1 and b2 with integer 2 as data.

WritePolicy wPolicy = new WritePolicy();

Bin b1 = new Bin("b1", Value.get("val1"));
Bin b2 = new Bin("b2", Value.get(2));

client.put(wPolicy, key, b1, b2);

##### About Jupyter Notebook Cell Execution sequence

If a previous code cell was executed, objects instantiated in it are available in the next cell that you execute. 

You can execute individual cells in any order - this is interactive execution.

Order of execution will determine the net result.  i.e. you can go back and forth between the record insertion cell above, and record read cell below. 

![fig_exec](./graphics/JupNb_Execution.png)

If you edit the insertion values, you must re-execute the write cell to put the new value on the Aerospike Server. Then, if you execute the read cell, you will see the updated value.

**_If you have lost track about where you are in the sequence, go to the top Menu, under Cell, Choose "Run All" to run all cells - top to bottom - in this notebook's sequence._** If you want to clear all objects in the kernel and restart from a fresh state, use **Kernel -> Restart & Run All**.


In [142]:
//Read the record that we just inserted

Record record = client.get(null, key);
System.out.println("Read back the record.");

System.out.println("Record values are:");
System.out.println(record);

Read back the record.
Record values are:
(gen:1),(exp:406920559),(bins:(b1:val1),(b2:2))


## Getting Back to Secondary Indexes in ver 6.1
Now that we are all setup, let us get back to discussing the new features in ver 6.1 Secondary Indexes.  

## Import Modules
Let us start by importing all the modules we will need for the SI examples.

In [143]:
// import all needed modules
import java.util.ArrayList;
import java.util.Arrays;
import java.util.HashMap;
import java.util.List;
import java.util.Map;

import com.aerospike.client.AerospikeException;
import com.aerospike.client.Key;
import com.aerospike.client.Bin;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.policy.ClientPolicy;
import com.aerospike.client.Record;
import com.aerospike.client.Operation;
import com.aerospike.client.Value;
import com.aerospike.client.Value.ListValue;
import com.aerospike.client.Value.MapValue;
import com.aerospike.client.cdt.ListOperation;
import com.aerospike.client.cdt.ListPolicy;
import com.aerospike.client.cdt.ListOrder;
import com.aerospike.client.cdt.ListWriteFlags;
import com.aerospike.client.cdt.MapOperation;
import com.aerospike.client.cdt.MapPolicy;
import com.aerospike.client.cdt.MapOrder;
import com.aerospike.client.cdt.MapWriteFlags;
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.query.RecordSet;
import com.aerospike.client.Record;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.query.IndexType;
import com.aerospike.client.query.IndexCollectionType;
import com.aerospike.client.task.IndexTask;
import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;
import com.aerospike.client.cdt.CTX;

## Define Constants and Helper Functions
Define constants for the namespaces *test*, set *cdt-indexing*, and helper functions _createIndex_, _dropIndex_, _executeQueryAndPrintResults_, and _truncateTestData_.

In [144]:
Statement stmt = new Statement();
    stmt.setNamespace("test");
    stmt.setSetName("demo");
    String location="{ \"type\": \"Point\", \"coordinates\": [-0.99999999, -0.99999999 ] }";
    Filter filter = Filter.geoContains("gj", location);
    stmt.setFilter(filter);
    //stmt.setBinNames("gj");
    RecordSet rs = client.query(null, stmt);
    while (rs.next()) {
        Key key = rs.getKey();
        Record record = rs.getRecord();
        System.out.format("key=%s bins=%s\n", key.userKey, record.bins);
    }
    //System.out.println();
    rs.close();

EvalException: Error -11,6,0,30000,0,5: Max retries exceeded: 5
sub-exceptions:
Error 201,1,0,30000,0,5,A1 127.0.0.1 3000: Index not found
Error 201,2,0,30000,0,5,A1 127.0.0.1 3000: Index not found
Error 201,3,0,30000,0,5,A1 127.0.0.1 3000: Index not found
Error 201,4,0,30000,0,5,A1 127.0.0.1 3000: Index not found
Error 201,5,0,30000,0,5,A1 127.0.0.1 3000: Index not found
Error 201,6,0,30000,0,5,A1 127.0.0.1 3000: Index not found


In [145]:
final String Namespace = "test";
final String Set = "cdt-indexing";
final String ListBin = "list_bin";
final String MapBin = "map_bin";


// convenience function to create an index - essentially a pass-through to the client API
void createIndex(String idxName, String binName, IndexType idxType, IndexCollectionType colType, CTX... ctx) {
    try {
            IndexTask task = client.createIndex(null,
                                       Namespace,
                                       Set,
                                       idxName,
                                       binName,
                                       idxType,
                                       colType,
                                       ctx);
            task.waitTillComplete(1000, 0);
        }
        catch (AerospikeException ae) {
            if (ae.getResultCode() != ResultCode.INDEX_ALREADY_EXISTS) {
                throw ae;
            }
        } 
        System.out.format("Created index %s on ns=%s set=%s bin=%s.\n", 
                                    idxName, Namespace, Set, binName);
}

// convenience function to drop an index - essentially a pass-through to the client API
void dropIndex(String idxName) {
    try {
        IndexTask task = client.dropIndex(null, Namespace, Set, idxName);
    }
    catch (AerospikeException ae) {
        if (ae.getResultCode() != ResultCode.INDEX_NOTFOUND) {
            throw ae;
        }
    } 
    System.out.format("Dropped index %s.\n", idxName);
}

// convenience function to execute a query using the input filter and print the results
void executeQueryAndPrintResults(Filter filter, String binName) {
    Statement stmt = new Statement();
    stmt.setNamespace(Namespace);
    stmt.setSetName(Set);
    stmt.setFilter(filter);
    stmt.setBinNames(binName);
    RecordSet rs = client.query(null, stmt);
    while (rs.next()) {
        Key key = rs.getKey();
        Record record = rs.getRecord();
        System.out.format("key=%s bins=%s\n", key.userKey, record.bins);
    }
    //System.out.println();
    rs.close();
}

// convenience function to truncate test data
void truncateTestData() {
    try {
        client.truncate(null, Namespace, null, null);
    }
    catch (AerospikeException e) {
        // ignore
    }
}

## CDT Test Data
We will illustrate how to create indexes on CDTs and issues queries using two CDTs - a nested List and a nested Map that are populated in 10 records each.

A nested List in records with user_key (id) 1-10. The list is held in bin list_bin and has the following numeric, string, and List elements.



Numeric elements: user_key to user_key+4.
For example, for user_key=1: 1, 2, 3, 4, 5.
String elements: "s"+user_key to "s"+(user_key+4).
For example, for user_key=1: "s1", "s2", "s3", "s4", "s5"
Nested List element: holds numeric elements (10\*user_key)+1 to (10\*user_key)+5.

For example, for user_key=1: [11, 12, 13, 44, 15].
So, the value in list_bin for the record with user_key=1 looks like:

[ 1, 2, 3, 4, 5, s1", "s2", "s3", "s4", "s5", [ 11, 12, 13, 44, 15 ] ].

![fig_CDTDM_1](./graphics/CDT_DM_1.png)

A nested Map in records with user_key (id) 101-110. The map is held in bin map_bin and has the following numeric, string, and List elements. Below, we have used i with a shorthand for i=user_key-100.


"oid": i.
For example, for i=1: 1.
"obj": a nested map with the following key-value pairs: {"attr1": (10\*i)+1, "attr2": "s"+(10\*i)+2, "subobj": {"attr3": (100\*i)+1, "attr4": (100\*i)+2}}.
For example, for i=1: {"attr1": 11, "attr2": "s12", "subobj": {"attr3": 101, "attr4": 102}}.
i: ["s"+i, "s"+i+1].

For example, for i=1: the key-value pair is 1: ["s1", "s2"].
So, the value in map_bin for the record with user_key=1 looks like:

{
  "oid": 1, 
  "obj": {"attr1": 11, "attr2": "s12", "subobj": {"attr3": 101, "attr4": 102}}, 
  1: ["s1", "s2"]
}

![fig_DM_2](./graphics/CDT_DM_2.png)


## Populate Test Data
Execute the cell below to populate the test data described above.

**We will insert 10 records with userkey=1 through 10 with ListBin, and  records with userkey = 101 thru 110 with MapBin.**


In [146]:
WritePolicy wpolicy = new WritePolicy();
wpolicy.sendKey = true;

// create the list_bin with [i..i+4, "s"+i.."s"+4, [10*i+1..10*i+5]]
for (int user_key = 1; user_key <= 10; user_key++) {
    Key key = new Key(Namespace, Set, user_key);

    // append integer and string elements
    ArrayList<Value> list_val = new ArrayList<Value>();
    for (int n = user_key; n < user_key+5; n++) {
        list_val.add(Value.get(n));
        list_val.add(Value.get("s"+Integer.toString(n)));
    }

    // append string elements
    //for (int n=user_key; n < user_key+5; n++) {
    //    list_val.add(Value.get("s"+Integer.toString(n)));
    //}

    // append the nested list element
    ArrayList<Integer> list_int = new ArrayList<Integer>();
    for (int n=user_key*10+1; n < user_key*10+6; n++) {
        list_int.add(n);
    }
    ListValue list_int_val = new ListValue(list_int);

    // create the list bin
    Record record = client.operate(wpolicy, key,
                                    ListOperation.clear(ListBin), 
                                    ListOperation.appendItems(ListBin, list_val), 
                                    ListOperation.append(ListBin, list_int_val));
}

// create map bin with 
//    "oid": i,
//    "obj": {"attr1": 10*i+1, "attr2": "s"+10*+i+2, "subobj": {"attr3": 100*1+1, "attr4": 100*1+2}}, 
//    i: ["s"+i, "s"+i+1]


for (int user_key = 101; user_key <= 110; user_key++) {
    Integer i = user_key - 100;
    
    Key key = new Key(Namespace, Set, user_key);
    
    // construct obj map value
    HashMap <String, Value> obj = new HashMap <String, Value>();
    obj.put("attr1", Value.get(10*i+1));    
    obj.put("attr2", Value.get("s"+Integer.toString(10*i+2)));   
    HashMap <String, Integer> subobj = new HashMap <String, Integer>();
    subobj.put("attr3", 100*i+1);    
    subobj.put("attr4", 100*i+2);    
    obj.put("subobj", new MapValue(subobj));   

    // construct arr list value
    ArrayList<String> arr = new ArrayList<String>();
    arr.add("s"+Integer.toString(i*10+1));
    arr.add("s"+Integer.toString(i*10+2));
    
    // create the map in map_bin
    MapPolicy mPolicy = new MapPolicy(MapOrder.UNORDERED, MapWriteFlags.DEFAULT);
    Record record = client.operate(wpolicy, key,
                                MapOperation.clear(MapBin), 
                                MapOperation.put(mPolicy, MapBin, 
                                            Value.get("oid"), Value.get(i)),
                                MapOperation.put(mPolicy, MapBin, 
                                            Value.get("obj"), new MapValue(obj)),
                                MapOperation.put(mPolicy, MapBin, 
                                            Value.get(i), new ListValue(arr))
                                ); 
}

## Examine Test Data
View the test data by executing the following commands in a Jupyter terminal tab.

**aql -c "select list_bin from test.cdt-indexing"**

**aql -c "select map_bin from test.cdt-indexing"**

## Non-Collection Indexes
We will illustrate how indexes on non-collection elements are created and used. We will start with results in mind, and then create the appropriate index, issue a query using the index, and show the results.

### Equality queries
Get records with a specific integer or string value at a specific index, rank, or key position of a List or a Map.

#### Records with a specific value at index X of a List or Map
##### The top level List in list_bin has has numeric value = 3 at List Index = 2.

![fig_EX_1](./graphics/CDT_EX_1.png)


In [147]:
// Create an index for element at position 2 of the top List in list_bin. 
// idx_list_bin_top_pos_2_num
createIndex("idx_list_bin_top_pos_2_num", ListBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, CTX.listIndex(2));

// issue the query and print results
System.out.println("Records with the value 3 at position 2 of the top List in list_bin:");
Filter filter = Filter.equal(ListBin, 3, CTX.listIndex(2));
executeQueryAndPrintResults(filter, ListBin);

Created index idx_list_bin_top_pos_2_num on ns=test set=cdt-indexing bin=list_bin.
Records with the value 3 at position 2 of the top List in list_bin:
key=2 bins={list_bin=[2, s2, 3, s3, 4, s4, 5, s5, 6, s6, [21, 22, 23, 24, 25]]}


##### The top level Map in map_bin has numeric value 7 at map index = 2.


![fig_EX_2](./graphics/CDT_EX_2.png)

In [148]:
// Create an index for element at position 2 of the top Map in map_bin. Note, in the key ordered sequence, 
//   "oid" element is in position 2 after the numeric key and "obj".
// idx_map_bin_top_pos_2_num
createIndex("idx_map_bin_top_pos_2_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, CTX.mapIndex(2));

// issue the query and print results
System.out.println("Records with the value 7 at position 2 of the top Map in map_bin:");
Filter filter = Filter.equal(MapBin, 7, CTX.mapIndex(2));
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_top_pos_2_num on ns=test set=cdt-indexing bin=map_bin.
Records with the value 7 at position 2 of the top Map in map_bin:
key=107 bins={map_bin={obj={attr2=s72, attr1=71, subobj={attr4=702, attr3=701}}, oid=7, 7=[s71, s72]}}


### Records with a specific value at rank X of a List or Map
##### The nested List in list_bin has numeric value = 35 in rank -1 (highest value).

![fig_EX_3](./graphics/CDT_EX_3.png)

In [149]:
// Create an index for element at rank -1 of the nested List in list_bin. Note the nested List is at the 
//   last poistion (index -1) in the top List.
// idx_list_bin_nested_rnk_-1_num
createIndex("idx_list_bin_nested_rnk_-1_num", ListBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
            CTX.listIndex(-1), CTX.listRank(-1));

// issue the query and print results
System.out.println("Records with the value 35 in rank -1 (highest value) in the nested List in list_bin:");
Filter filter = Filter.equal(ListBin, 35, CTX.listIndex(-1), CTX.listRank(-1));
executeQueryAndPrintResults(filter, ListBin);

Created index idx_list_bin_nested_rnk_-1_num on ns=test set=cdt-indexing bin=list_bin.
Records with the value 35 in rank -1 (highest value) in the nested List in list_bin:
key=3 bins={list_bin=[3, s3, 4, s4, 5, s5, 6, s6, 7, s7, [31, 32, 33, 34, 35]]}


##### The nested "subobj" Map in map_bin has numeric value = 901 in rank 0 (lowest value in all map values)
![fig_EX_4](./graphics/CDT_EX_4.png)

In [150]:
// Create an index for element at rank 0 (lowest value) in the nested "subobj" Map in map_bin. 
// idx_map_bin_subobj_rnk_0_num
createIndex("idx_map_bin_subobj_rnk_0_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")), CTX.mapRank(0));

// issue the query and print results
System.out.println("Records with the value 901 in rank 0 (lowest value) in the nested \"subobj\" Map in map_bin:");
Filter filter = Filter.equal(MapBin, 901, 
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")), CTX.mapRank(0));
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_subobj_rnk_0_num on ns=test set=cdt-indexing bin=map_bin.
Records with the value 901 in rank 0 (lowest value) in the nested "subobj" Map in map_bin:
key=109 bins={map_bin={9=[s91, s92], obj={attr2=s92, attr1=91, subobj={attr4=902, attr3=901}}, oid=9}}


##### The nested "subobj" Map in map_bin has numeric value = 901 in rank 0 (lowest value in all map values)
![fig_EX_4](./graphics/CDT_EX_4.png)

In [169]:
// Create an index for mapkeys in for nested map at "obj" Map in map_bin. 
// idx_map_bin_subobj_rnk_0_num
createIndex("idx_map_bin_obj_mapkeys", MapBin, IndexType.STRING, IndexCollectionType.MAPKEYS, 
            CTX.mapKey(Value.get("obj")));

// issue the query and print results
System.out.println("Records with mapkey value \"subobj\" Map in map_bin key \"obj\":");
Filter filter = Filter.contains(MapBin,  IndexCollectionType.MAPKEYS,"subobj",
            CTX.mapKey(Value.get("obj")));
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_obj_mapkeys on ns=test set=cdt-indexing bin=map_bin.
Records with mapkey value "subobj" Map in map_bin key "obj":


#### Records with a specific value at key X of a Map
##### The top level Map in map_bin has numeric value = 8 at key "oid".
![fig_EX_5](./graphics/CDT_EX_5.png)

In [151]:
// Create an index for element with key "oid" in the top level Map in map_bin. 
// idx_map_bin_top_key_oid_num
createIndex("idx_map_bin_top_key_oid_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
            CTX.mapKey(Value.get("oid")));

// issue the query and print results
System.out.println("Records with the value 8 at key \"oid\" in the top level Map in map_bin:");
Filter filter = Filter.equal(MapBin, 8, 
            CTX.mapKey(Value.get("oid")));
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_top_key_oid_num on ns=test set=cdt-indexing bin=map_bin.
Records with the value 8 at key "oid" in the top level Map in map_bin:
key=108 bins={map_bin={8=[s81, s82], obj={attr2=s82, attr1=81, subobj={attr4=802, attr3=801}}, oid=8}}


##### The nested "obj" Map in map_bin has string value = "s42" at map key = "attr2".
![fig_EX_6](./graphics/CDT_EX_6.png)

In [152]:
// Create an index for element with key "attr2" in the nested "obj" Map in map_bin. 
// idx_map_bin_obj_key_attr2_str
createIndex("idx_map_bin_obj_key_attr2_str", MapBin, IndexType.STRING, IndexCollectionType.DEFAULT, 
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("attr2")));

// issue the query and print results
System.out.println("Records with the value \"s42\" at key \"attr2\" in the nested \"obj\" Map in map_bin:");
Filter filter = Filter.equal(MapBin, "s42", 
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("attr2")));
executeQueryAndPrintResults(filter, MapBin);


Created index idx_map_bin_obj_key_attr2_str on ns=test set=cdt-indexing bin=map_bin.
Records with the value "s42" at key "attr2" in the nested "obj" Map in map_bin:
key=104 bins={map_bin={4=[s41, s42], obj={attr2=s42, attr1=41, subobj={attr4=402, attr3=401}}, oid=4}}


### Range Queries
Get records having an integer value within a range at a specific index, rank, or key position of a List or a Map. Range queries are supported on integer values only.

We will use the indexes defined above for the range queries too.

#### Records with value in s specific range at index X rank position of a List or Map
##### The top level List in list_bin numeric value is in range 3-5 at List Index = 2
![fig_EX_7](./graphics/CDT_EX_1.png)

In [153]:
// issue the query and print results
// createIndex("idx_list_bin_top_pos_2_num", ListBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
//              CTX.listIndex(2)); 
// already created this SI

System.out.println("Records with value in range 3-5 at position 2 in the top level List in list_bin:");
Filter filter = Filter.range(ListBin, 3, 5, CTX.listIndex(2));
executeQueryAndPrintResults(filter, ListBin);

Records with value in range 3-5 at position 2 in the top level List in list_bin:
key=3 bins={list_bin=[3, s3, 4, s4, 5, s5, 6, s6, 7, s7, [31, 32, 33, 34, 35]]}
key=4 bins={list_bin=[4, s4, 5, s5, 6, s6, 7, s7, 8, s8, [41, 42, 43, 44, 45]]}
key=2 bins={list_bin=[2, s2, 3, s3, 4, s4, 5, s5, 6, s6, [21, 22, 23, 24, 25]]}


##### The top level Map in map_bin has numeric value in range 5 - 7 at map index = 2.
![fig_EX_8](./graphics/CDT_EX_8.png)

In [154]:
// issue the query and print results
// createIndex("idx_map_bin_top_pos_2_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, CTX.mapIndex(2));  
// already created this SI
System.out.println("Records with value in range 5-7 at position 2 of the top Map in map_bin:");
Filter filter = Filter.range(MapBin, 5, 7, CTX.mapIndex(2));
executeQueryAndPrintResults(filter, MapBin);

Records with value in range 5-7 at position 2 of the top Map in map_bin:
key=107 bins={map_bin={obj={attr2=s72, attr1=71, subobj={attr4=702, attr3=701}}, oid=7, 7=[s71, s72]}}
key=106 bins={map_bin={obj={attr2=s62, attr1=61, subobj={attr4=602, attr3=601}}, 6=[s61, s62], oid=6}}
key=105 bins={map_bin={5=[s51, s52], obj={attr2=s52, attr1=51, subobj={attr4=502, attr3=501}}, oid=5}}


#### Records with value in s specific range at rank X of a List or Map
##### The nested List in list_bin has numeric value in range 20 – 50 in rank -1 (highest value).
![fig_EX_9](./graphics/CDT_EX_3.png)

In [155]:
// issue the query and print results
// createIndex("idx_list_bin_nested_rnk_-1_num", ListBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
//              CTX.listIndex(-1), CTX.listRank(-1));
// already created this SI
System.out.println("Records with value in range 20-50 in rank -1 (highest value) in the nested List in list_bin:");
Filter filter = Filter.range(ListBin, 20, 60, CTX.listIndex(-1), CTX.listRank(-1));
executeQueryAndPrintResults(filter, ListBin);

Records with value in range 20-50 in rank -1 (highest value) in the nested List in list_bin:
key=3 bins={list_bin=[3, s3, 4, s4, 5, s5, 6, s6, 7, s7, [31, 32, 33, 34, 35]]}
key=4 bins={list_bin=[4, s4, 5, s5, 6, s6, 7, s7, 8, s8, [41, 42, 43, 44, 45]]}
key=2 bins={list_bin=[2, s2, 3, s3, 4, s4, 5, s5, 6, s6, [21, 22, 23, 24, 25]]}
key=5 bins={list_bin=[5, s5, 6, s6, 7, s7, 8, s8, 9, s9, [51, 52, 53, 54, 55]]}


##### The nested "subobj" Map in map_bin has numeric value in range 500 – 800 in rank 0 (lowest value in all map values).
![fig_EX_10](./graphics/CDT_EX_4.png)

In [156]:
// issue the query and print results
// createIndex("idx_map_bin_subobj_rnk_0_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, 
//              CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")), CTX.mapRank(0));  
// already created this SI

System.out.println("Records with value in range 500-800 in rank 0 (lowest value) in the nested \"subobj\" Map in map_bin:");
Filter filter = Filter.range(MapBin, 500, 801,
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")), CTX.mapRank(0));
executeQueryAndPrintResults(filter, MapBin);

Records with value in range 500-800 in rank 0 (lowest value) in the nested "subobj" Map in map_bin:
key=108 bins={map_bin={8=[s81, s82], obj={attr2=s82, attr1=81, subobj={attr4=802, attr3=801}}, oid=8}}
key=106 bins={map_bin={obj={attr2=s62, attr1=61, subobj={attr4=602, attr3=601}}, 6=[s61, s62], oid=6}}
key=105 bins={map_bin={5=[s51, s52], obj={attr2=s52, attr1=51, subobj={attr4=502, attr3=501}}, oid=5}}
key=107 bins={map_bin={obj={attr2=s72, attr1=71, subobj={attr4=702, attr3=701}}, oid=7, 7=[s71, s72]}}


#### Records with value in s specific range at key X of a Map

##### The top level Map in map_bin has numeric value in range 4 - 6 at key "oid".
![fig_EX_11](./graphics/CDT_EX_5.png)


In [157]:
// issue the query and print results
// createIndex("idx_map_bin_top_key_oid_num", MapBin, IndexType.NUMERIC, IndexCollectionType.DEFAULT, CTX.mapKey(Value.get("oid")));  
// already created this SI
System.out.println("Records with value in range 4-6 at key \"oid\" in the top level Map in map_bin:");
Filter filter = Filter.range(MapBin, 4, 6, CTX.mapKey(Value.get("oid")));
executeQueryAndPrintResults(filter, MapBin);

Records with value in range 4-6 at key "oid" in the top level Map in map_bin:
key=104 bins={map_bin={4=[s41, s42], obj={attr2=s42, attr1=41, subobj={attr4=402, attr3=401}}, oid=4}}
key=106 bins={map_bin={obj={attr2=s62, attr1=61, subobj={attr4=602, attr3=601}}, 6=[s61, s62], oid=6}}
key=105 bins={map_bin={5=[s51, s52], obj={attr2=s52, attr1=51, subobj={attr4=502, attr3=501}}, oid=5}}


## Collection Indexes
Collection indexes are defined on List and Map elements with the collection type LIST (list values in the List), MAPKEYS (key values in the Map), or MAPVALUES (values in the Map). The indexes can be used for equality queries on integr and string elements, as well as range queries on integer elements. We will illustrate these variations below.

### Equality Queries
Get records with a specific integer or string value in a List or Map.

#### Records with a List or Map containing a specific value.

##### The nested List in map_bin at MapIndex = 0 contains "s91".
![fig_EX_12](./graphics/CDT_EX_7.png)

In [158]:
// Create an index for the nested List in map_bin. Note, in key-ordered sequence, the numeric key is the first one.
// idx_map_bin_i_list_str
createIndex("idx_map_bin_i_list_str", MapBin, IndexType.STRING, IndexCollectionType.LIST, CTX.mapIndex(0));

// issue the query and print results
System.out.println("Records with the value \"s91\" in the nested list \"arr\" in map_bin:");
Filter filter = Filter.contains(MapBin, IndexCollectionType.LIST, "s91", CTX.mapIndex(0));
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_i_list_str on ns=test set=cdt-indexing bin=map_bin.
Records with the value "s91" in the nested list "arr" in map_bin:
key=109 bins={map_bin={9=[s91, s92], obj={attr2=s92, attr1=91, subobj={attr4=902, attr3=901}}, oid=9}}


##### The top level Map in map_bin contains a key whose numeric value = 5.
![fig_EX_13](./graphics/CDT_EX_9.png)

In [159]:
// Create an index for keys in top level Map in map_bin. 
// idx_map_bin_top_mapkeys_num
createIndex("idx_map_bin_top_mapkeys_num", MapBin, IndexType.NUMERIC, IndexCollectionType.MAPKEYS);

// issue the query and print results
System.out.println("Records with key 5 in the top level Map in map_bin:");
Filter filter = Filter.contains(MapBin, IndexCollectionType.MAPKEYS, 5);
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_top_mapkeys_num on ns=test set=cdt-indexing bin=map_bin.
Records with key 5 in the top level Map in map_bin:
key=105 bins={map_bin={5=[s51, s52], obj={attr2=s52, attr1=51, subobj={attr4=502, attr3=501}}, oid=5}}


##### The nested "obj" Map in map_bin contains a value "s12".
![fig_EX_14](./graphics/CDT_EX_10.png)

In [160]:
// Create an index for values in nested Map "obj" in map_bin. 
// idx_map_bin_obj_mapvals_str
createIndex("idx_map_bin_obj_mapvals_str", MapBin, IndexType.STRING, IndexCollectionType.MAPVALUES, 
            CTX.mapKey(Value.get("obj")));

// issue the query and print results
System.out.println("Records with the value \"s12\" in the nested Map \"obj\" in map_bin:");
Filter filter = Filter.contains(MapBin, IndexCollectionType.MAPVALUES, "s22", CTX.mapKey(Value.get("obj")));
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_obj_mapvals_str on ns=test set=cdt-indexing bin=map_bin.
Records with the value "s12" in the nested Map "obj" in map_bin:
key=102 bins={map_bin={obj={attr2=s22, attr1=21, subobj={attr4=202, attr3=201}}, 2=[s21, s22], oid=2}}


### Range Queries
Get records with a range of integer values in a List or Map.

#### Records with a List or Map containing an integer in a given range.

##### The nested list in list_bin has a value in the numeric value range 85-93.
![fig_EX_15](./graphics/CDT_EX_11.png)

In [161]:
// Create an index for the nested List in map_bin. Note, the nested list is the last element.
// idx_list_bin_nested_list_num
createIndex("idx_list_bin_nested_list_num", ListBin, IndexType.NUMERIC, IndexCollectionType.LIST, 
            CTX.listIndex(-1));

// issue the query and print results
System.out.println("Records with value in range 85-93 in the nested list in list_bin:");
Filter filter = Filter.range(ListBin, IndexCollectionType.LIST, 85, 93, CTX.listIndex(-1));
executeQueryAndPrintResults(filter, ListBin);

Created index idx_list_bin_nested_list_num on ns=test set=cdt-indexing bin=list_bin.
Records with value in range 85-93 in the nested list in list_bin:
key=8 bins={list_bin=[8, s8, 9, s9, 10, s10, 11, s11, 12, s12, [81, 82, 83, 84, 85]]}
key=9 bins={list_bin=[9, s9, 10, s10, 11, s11, 12, s12, 13, s13, [91, 92, 93, 94, 95]]}


##### The top level Map in map_bin has a MapKey in range 5-7.
![fig_EX_16](./graphics/CDT_EX_9.png)

In [162]:
// use the index created earlier
// createIndex("idx_map_bin_top_mapkeys_num", MapBin, IndexType.NUMERIC, IndexCollectionType.MAPKEYS);  
// issue the query and print results
System.out.println("Records with key in range 5-7 in the top level Map in map_bin:");
Filter filter = Filter.range(MapBin, IndexCollectionType.MAPKEYS, 5, 7);
executeQueryAndPrintResults(filter, MapBin);

Records with key in range 5-7 in the top level Map in map_bin:
key=106 bins={map_bin={obj={attr2=s62, attr1=61, subobj={attr4=602, attr3=601}}, 6=[s61, s62], oid=6}}
key=107 bins={map_bin={obj={attr2=s72, attr1=71, subobj={attr4=702, attr3=701}}, oid=7, 7=[s71, s72]}}
key=105 bins={map_bin={5=[s51, s52], obj={attr2=s52, attr1=51, subobj={attr4=502, attr3=501}}, oid=5}}


##### The nested Map "subobj" in map_bin has MapValue(s) in numeric value in range of 300 – 500.
![fig_EX_17](./graphics/CDT_EX_4.png)

In [163]:
// Create an index for values in nested Map "subobj" in map_bin. 
// idx_map_bin_subobj_mapvals_num
createIndex("idx_map_bin_subobj_mapvals_num", MapBin, IndexType.NUMERIC, IndexCollectionType.MAPVALUES, 
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")));

// issue the query and print results
System.out.println("Records with value in range 300-500 in the nested Map \"subobj\" in map_bin:");
Filter filter = Filter.range(MapBin, IndexCollectionType.MAPVALUES, 302, 501, 
            CTX.mapKey(Value.get("obj")), CTX.mapKey(Value.get("subobj")));
executeQueryAndPrintResults(filter, MapBin);

Created index idx_map_bin_subobj_mapvals_num on ns=test set=cdt-indexing bin=map_bin.
Records with value in range 300-500 in the nested Map "subobj" in map_bin:
key=103 bins={map_bin={obj={attr2=s32, attr1=31, subobj={attr4=302, attr3=301}}, oid=3, 3=[s31, s32]}}
key=104 bins={map_bin={4=[s41, s42], obj={attr2=s42, attr1=41, subobj={attr4=402, attr3=401}}, oid=4}}
key=105 bins={map_bin={5=[s51, s52], obj={attr2=s52, attr1=51, subobj={attr4=502, attr3=501}}, oid=5}}


## Clean up

Run in last code cell: %sh aql -h 127.0.0.1 -f "./aqlScripts/truncate_test.aql"

In [164]:
import com.aerospike.client.exp.Expression;
Expression ep = new Expression();
System.out.println(ep.getBase64("liKkA29iaiKnA3N1Ym9iaiEA"));

CompilationException: 